In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from calculator import Methods,Calculator
from const import Const
from test1 import Test

In [2]:
cal=Calculator()
cal.read_excel('Data2.xlsx')
te=Test()

In [3]:
# M_v=8e+6，固定第一个的内边界g/r的值，然后求出新的ST
# 先取出第一个内边界的g/r值
filename='store-all.csv'
f=open(filename,'r')
values=[]
for line in f:
    values.append(line.split(','))
g_in=float(values[1][-1])
f.close()

#再取出用作参考的ST的数据，后面用order索引
"""filename='../Data/store-3.3e+3.csv'
f=open(filename,'r')
values=[]
for line in f:
    values.append(line.split(','))
f.close()"""

"filename='../Data/store-3.3e+3.csv'\nf=open(filename,'r')\nvalues=[]\nfor line in f:\n    values.append(line.split(','))\nf.close()"

In [4]:
result=np.ones((len(cal.M_p),6))
maxtime=20
for order in range(len(cal.M_p)):
    m=cal.M_p[order]
    L_s=cal.L_init[order]
    ST=cal.ST[order]
    te.con.set_M_p(m,8e+6)
    x_start=-3.0e+4
    x_step1,x_step2=0.5e+3,0.1e+3
    error=1e-6
    y_start=te.cal_ML_simple_B(ST,L_s,True,0.0,x_start)[-1]/(te.r[-1]*te.con.R_B)
    y_target=g_in
    y1=y_start
    x1=x_start
    i=1
    while abs(y1-y_target)>error:
        if i>1:
            x2=x1+x_step2
        else:
            x2=x1+x_step1
        y2=te.cal_ML_simple_B(ST,L_s,True,0.0,x2)[-1]/(te.r[-1]*te.con.R_B)
        grad=(y2-y1)/(x2-x1)
        x1=(y_target-y2)/grad+x2 
        y1=te.cal_ML_simple_B(ST,L_s,True,0.0,x1)[-1]/(te.r[-1]*te.con.R_B)
        i+=1
        if i>maxtime:
            print('come out.')
            break
    dgr=x1
    x_start=ST 
    x_step=ST/10
    error=1e-5
    y_start=te.cal_ML_simple_B(x_start,L_s,True,0.0,dgr)[-2]
    y_target=0.0
    y1=y_start
    x1=x_start
    i=1
    while abs(y1-y_target)>error:
        x2=x1+x_step
        y2=te.cal_ML_simple_B(x2,L_s,True,0.0,dgr)[-2]
        grad=(y2-y1)/(x2-x1)
        x1=(y_target-y2)/grad+x2 
        y1=te.cal_ML_simple_B(x1,L_s,True,0.0,dgr)[-2]
        i+=1
        if i>maxtime:
            print('come out.')
            break
    ST=x1
    te.cal_ML_simple_B(ST,L_s,True,0.0,dgr)
    result[order]=[te.P[-1],te.T[-1],te.L[0],ST,m,dgr]
    print('{0} has been done.'.format(order))

0 has been done.
1 has been done.
2 has been done.
/home/xiaoliang/miniconda3/envs/vscode/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/xiaoliang/miniconda3/envs/vscode/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
/home/xiaoliang/miniconda3/envs/vscode/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/xiaoliang/Project/Test/test1.py:56: RuntimeWarning: overflow encountered in power
  judge=(cls.con.c_T*1e24*abs(L)*P**(cls.con.alpha+1)*T**(cls.con.beta-4)/M)<(np.ones(len(L))*cls.con.g_ad)
/home/xiaoliang/Project/Test/test1.py:56: RuntimeWarning: invalid val

In [1]:
result

NameError: name 'result' is not defined

In [5]:
f=open('store-ST-dgr.csv','w')
f.write('order,P,T,L,ST,m,dgr,M_v=8e+6\n')
for order in range(len(result)):
    f.write('{0},{1},{2},{3},{4},{5},{6}\n'.format(order,result[order][0],result[order][1],result[order][2],result[order][3],result[order][4],result[order][5]))
f.close()

In [7]:
result[0]

array([ 8.08804232e+05,  3.12841718e+01,  3.52440268e+03,  2.34526138e-02,
        5.15000000e+00, -3.00000000e+03])

In [8]:
g_in

0.15602485393910218